In [1]:
NAME = "change the conv2d"

In [2]:
BATCH_SIZE = 32

In [3]:
import os
import cv2
import torch
import numpy as np

In [4]:
def load_data(img_size=112):
    data = []
    index = -1
    labels = {}
    for directory in os.listdir('./data/'):
        index += 1
        labels[f'./data/{directory}/'] = [index,-1]
    print(len(labels))
    for label in labels:
        for file in os.listdir(label):
            filepath = label + file
            img = cv2.imread(filepath,cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img,(img_size,img_size))
            img = img / 255.0
            data.append([
                np.array(img),
                labels[label][0]
            ])
            labels[label][1] += 1
    for _ in range(12):
        np.random.shuffle(data)
    print(len(data))
    np.save('./data.npy',data)
    return data

In [5]:
import torch

In [6]:
def other_loading_data_proccess(data):
    X = []
    y = []
    print('going through the data..')
    for d in data:
        X.append(d[0])
        y.append(d[1])
    print('splitting the data')
    VAL_SPLIT = 0.25
    VAL_SPLIT = len(X)*VAL_SPLIT
    VAL_SPLIT = int(VAL_SPLIT)
    X_train = X[:-VAL_SPLIT]
    y_train = y[:-VAL_SPLIT]
    X_test = X[-VAL_SPLIT:]
    y_test = y[-VAL_SPLIT:]
    print('turning data to tensors')
    X_train = torch.from_numpy(np.array(X_train))
    y_train = torch.from_numpy(np.array(y_train))
    X_test = torch.from_numpy(np.array(X_test))
    y_test = torch.from_numpy(np.array(y_test))
    return [X_train,X_test,y_train,y_test]

In [7]:
REBUILD_DATA = True
if REBUILD_DATA:
    data = load_data()
    np.random.shuffle(data)
    X_train,X_test,y_train,y_test = other_loading_data_proccess(data)

In [8]:
import torch
import torch.nn as nn

In [9]:
import torch.nn.functional as F

In [10]:
# class Test_Model(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(1, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 25 * 25, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 36)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(-1, 16 * 25 * 25)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

In [11]:
class Test_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv3 = nn.Conv2d(32,64,5)
        self.conv2 = nn.Conv2d(64, 128, 5)
        self.fc1 = nn.Linear(128 * 10 * 10, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128, 36)

    def forward(self, x,shape=False):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv2(x)))
        if shape:
            print(x.shape)
        x = x.view(-1, 128 * 10 * 10)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc4(x))
        x = self.fc3(x)
        return x

In [12]:
device = torch.device('cuda')

In [13]:
model = Test_Model().to(device)

In [14]:
preds = model(X_test.reshape(-1,1,112,112).float().to(device),True)

In [15]:
preds[0]

tensor([-0.0191,  0.0632,  0.0395,  0.0232,  0.0519,  0.0292, -0.0144, -0.0203,
        -0.0549, -0.0322,  0.0488,  0.0090,  0.0260,  0.0676, -0.0324, -0.0547,
        -0.0621,  0.0834, -0.0553, -0.0824,  0.0712, -0.0096,  0.0318, -0.0321,
         0.0182,  0.0848, -0.0889, -0.0406, -0.0251, -0.0363,  0.1071,  0.0589,
        -0.0466,  0.0857, -0.0178, -0.0514], device='cuda:0',
       grad_fn=<SelectBackward>)

In [16]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.1)
criterion = nn.CrossEntropyLoss()

In [17]:
EPOCHS = 5

In [18]:
loss_logs = []

In [19]:
from tqdm import tqdm

In [20]:
PROJECT_NAME = "Sign-Language-Recognition"

In [21]:
def test(net,X,y):
    correct = 0
    total = 0
    net.eval()
    with torch.no_grad():
        for i in range(len(X)):
            real_class = torch.argmax(y[i]).to(device)
            net_out = net(X[i].view(-1,1,112,112).to(device).float())
            net_out = net_out[0]
            predictied_class = torch.argmax(net_out)
            if predictied_class == real_class:
                correct += 1
            total += 1
    return round(correct/total,3)

In [22]:
import wandb

In [23]:
len(os.listdir('./data/'))

36

In [24]:
import random

In [25]:
# index = random.randint(0,29)
# print(index)
# wandb.init(project=PROJECT_NAME,name=NAME)
# for _ in tqdm(range(EPOCHS)):
#     for i in range(0,len(X_train),BATCH_SIZE):
#         X_batch = X_train[i:i+BATCH_SIZE].view(-1,1,112,112).to(device)
#         y_batch = y_train[i:i+BATCH_SIZE].to(device)
#         model.to(device)
#         preds = model(X_batch.float())
#         loss = criterion(preds,torch.tensor(y_batch,dtype=torch.long))
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         wandb.log({'loss':loss.item(),'accuracy':test(model,X_train,y_train)*100,'val_accuracy':test(model,X_test,y_test)*100,'pred':torch.argmax(preds[index]),'real':torch.argmax(y_batch[index])})
# wandb.finish()

In [26]:
import matplotlib.pyplot as plt

In [27]:
import pandas as pd

In [28]:
df = pd.Series(loss_logs)

In [29]:
df.plot.line(figsize=(12,6))

<AxesSubplot:>

In [30]:
test(model,X_test,y_test)

0.0

In [31]:
test(model,X_train,y_train)

0.0

In [32]:
preds

tensor([[-0.0191,  0.0632,  0.0395,  ...,  0.0857, -0.0178, -0.0514],
        [-0.0194,  0.0627,  0.0399,  ...,  0.0855, -0.0175, -0.0516],
        [-0.0196,  0.0631,  0.0394,  ...,  0.0856, -0.0178, -0.0506],
        ...,
        [-0.0193,  0.0633,  0.0396,  ...,  0.0854, -0.0180, -0.0509],
        [-0.0184,  0.0624,  0.0386,  ...,  0.0853, -0.0176, -0.0514],
        [-0.0195,  0.0630,  0.0395,  ...,  0.0859, -0.0184, -0.0514]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [33]:
X_testing = X_train
y_testing = y_train
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for i in range(len(X_testing)):
        real_class = torch.argmax(y_testing[i]).to(device)
        net_out = model(X_testing[i].view(-1,1,112,112).to(device).float())
        net_out = net_out[0]
        predictied_class = torch.argmax(net_out)
#         print(predictied_class)
        if str(predictied_class) == str(real_class):
            correct += 1
        total += 1
print(round(correct/total,3))

In [34]:
# for real,pred in zip(y_batch,preds):
#     print(real)
#     print(torch.argmax(pred))
#     print('\n')

In [35]:
# conv2d_output
# conv2d_1_ouput
# conv2d_2_ouput
# output_fc1
# output_fc2
# output_fc4
# max_pool2d_keranl
# max_pool2d
# num_of_linear

# activation
# best num of epochs
# best optimizer
# best loss
## best lr

In [36]:
class Test_Model(nn.Module):
    def __init__(self,conv2d_output=128,conv2d_1_ouput=32,conv2d_2_ouput=64,output_fc1=512,output_fc2=256,output_fc4=128,output=36,activation=F.relu,max_pool2d_keranl=2):
        super().__init__()
        print(conv2d_output)
        print(conv2d_1_ouput)
        print(conv2d_2_ouput)
        print(output_fc1)
        print(output_fc2)
        print(output_fc4)
        print(activation)
        self.conv2d_output = conv2d_output
        self.pool = nn.MaxPool2d(max_pool2d_keranl)
        self.conv1 = nn.Conv2d(1, conv2d_1_ouput, 5)
        self.conv3 = nn.Conv2d(conv2d_1_ouput,conv2d_2_ouput,5)
        self.conv2 = nn.Conv2d(conv2d_2_ouput, conv2d_output, 5)
        self.fc1 = nn.Linear(conv2d_output * 10 * 10, output_fc1)
        self.fc2 = nn.Linear(output_fc1, output_fc2)
        self.fc4 = nn.Linear(output_fc2,output_fc4)
        self.fc3 = nn.Linear(output_fc4, output)
        self.activation = activation

    def forward(self, x,shape=False):
        x = self.pool(self.activation(self.conv1(x)))
        x = self.pool(self.activation(self.conv3(x)))
        x = self.pool(self.activation(self.conv2(x)))
        if shape:
            print(x.shape)
        x = x.view(-1, self.conv2d_output * 10 * 10)
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc4(x))
        x = self.fc3(x)
        return x

In [37]:
# conv2d_output
# conv2d_1_ouput
# conv2d_2_ouput
# output_fc1
# output_fc2
# output_fc4
# max_pool2d_keranl
# max_pool2d
# num_of_linear
# best num of epochs
# best loss
## best lr
# batch size

In [38]:
EPOCHS = 3
BATCH_SIZE = 32

In [39]:
# activation = nn.SELU()
# best optimizer = torch.optim.Adadelta
# best loss = nn.CrossEntropyLoss
## best lr = 0.625
# conv2d_output = 
# conv2d_1_ouput = 

# conv2d_2_ouput
# output_fc1
# output_fc2
# output_fc4
# max_pool2d_keranl
# max_pool2d
# num_of_linear
# best num of epochs

In [40]:
def get_loss(criterion,y,model,X):
    preds = model(X.view(-1,1,112,112).to(device).float())
    preds.to(device)
    loss = criterion(preds,torch.tensor(y,dtype=torch.long).to(device))
    loss.backward()
    return loss.item()

In [41]:
conv2d_1_ouputs = [32,64,128,256,512]
for conv2d_1_ouput in conv2d_1_ouputs:
    model = Test_Model(activation=nn.SELU(),conv2d_1_ouput=conv2d_1_ouput)
    optimizer = torch.optim.Adadelta(model.parameters(),lr=0.625)
    criterion = nn.CrossEntropyLoss()
    wandb.init(project=PROJECT_NAME,name=f'conv2d_1_ouputs-{conv2d_1_ouput}')
    for _ in tqdm(range(EPOCHS)):
        for i in range(0,len(X_train),BATCH_SIZE):
            X_batch = X_train[i:i+BATCH_SIZE].view(-1,1,112,112).to(device)
            y_batch = y_train[i:i+BATCH_SIZE].to(device)
            model.to(device)
            preds = model(X_batch.float())
            loss = criterion(preds,torch.tensor(y_batch,dtype=torch.long))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            wandb.log({'loss':loss.item(),'accuracy':test(model,X_train,y_train)*100,'val_accuracy':test(model,X_test,y_test)*100,'val_loss':get_loss(criterion,y_test,model,X_test)})
        print(f'{torch.argmax(preds[1])} \n {y_batch[1]}')
        print(f'{torch.argmax(preds[2])} \n {y_batch[2]}')
        print(f'{torch.argmax(preds[3])} \n {y_batch[3]}')
        print(f'{torch.argmax(preds[4])} \n {y_batch[4]}')
    wandb.finish()
    torch.cuda.empty_cache()